In [ ]:
import h5py
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import gc
#import minpy.numpy as np
from minpy.context import gpu

%matplotlib inline

In [ ]:
def upload_all_files(dirName):
    files = []
    directory = os.fsencode(dirName)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        f = h5py.File(dirName + '/' + filename)
        files.append(f)
    return files

In [ ]:
files = upload_all_files("Dataset")

In [ ]:
def upload_file(fPath):
    f = h5py.File(fPath)
    return f

In [ ]:
def channel_labels():
    channels = []
    f = files[0]
    for i in range(69):
        l = len(f[h5py.h5r.get_name(f['cnt']['clab'][i][0], f.id)].value)
        channel = ''
        for j in range(l):
            channel += chr(f[h5py.h5r.get_name(f['cnt']['clab'][i][0], f.id)].value[j])
        #print(str(i) + ', ' + channel)
        channels.append(channel)
    return channels

In [ ]:
channels = channel_labels()

In [ ]:
def dataframe_fill(df, j):
    for i in range(len(df.columns)):
        df[df.columns[i]] = files[j]['cnt']['x'][i]
    return df

In [ ]:
def files_to_dfs():
    dfs = []
    for i in range(len(files)):
        dfs.append(pd.DataFrame(columns = channels))
        dataframe_fill(dfs[i], i)
        print(i)
    return dfs

In [ ]:
dfs = files_to_dfs()

In [ ]:
def drop_cols():
    for i in range(len(dfs)):
        cols = [0, 5, 61, 62, 63, 64, 65, 66, 67, 68]
        dfs[i].drop(dfs[i].columns[cols], axis=1, inplace=True)

In [ ]:
def events():
    eventsList = []
    for i in range(5):
        l = len(files[0][h5py.h5r.get_name(files[0]['mrk']['className'][i][0], files[0].id)].value)
        event = ''
        for j in range(l):
            event += chr(files[0][h5py.h5r.get_name(files[0]['mrk']['className'][i][0], files[0].id)].value[j])
        print(str(i) + ', ' + event)
        eventsList.append(event)
    return eventsList

In [ ]:
eventsList = events()

In [ ]:
def events_to_lists():
    ys = []
    for j in range(len(files)):
        ys.append([])
        for i in range(files[j]['mrk']['y'].shape[0]):
            evIndex = np.where(files[j]['mrk']['y'][i] == 1)[0][0]
            ys[j].append(eventsList[evIndex])
    return ys

In [ ]:
ys = events_to_lists()

In [ ]:
def timestamp_list():
    stamps = []
    for j in range (len(files)):
        stamps.append([])
        for i in range(len(files[j]['mrk']['time'])):
            stamps[j].append(files[j]['mrk']['time'][i][0])
    return stamps

In [ ]:
stamps = timestamp_list()

In [ ]:
def reacts_list():
    reacts = []
    for j in range (len(files)):
        reacts.append([])
        for i in range(len(files[j]['mrk']['event']['react'][0])):
            reacts[j].append(files[j]['mrk']['event']['react'][0][i])
    return reacts

In [ ]:
reacts = reacts_list()

In [ ]:
def make_columns():
    newCols = []
    for col in dfs[0].columns:
        for i in range(300):
            newCols.append(col + '.' + str(i))
    newCols.append('y')
    return newCols

In [ ]:
newCols = make_columns()

In [ ]:
def positive_events():
    #positive events
    #3oo cols for each electrode
    posDfs = []
    for i in range(len(dfs)):
        print("i: " + str(i))
        posDfs.append(pd.DataFrame(columns = newCols))
        for j in range (len(stamps[i])):
            if (ys[i][j] == "react_emg"):
                car_brake = int((stamps[i][j] - reacts[i][j])*0.2)
                if (car_brake+300 < len(dfs[i])):
                    posDfs[i].loc[len(posDfs[i])] = convert_to_row(car_brake, i, "brake")
                    #print(j)
        print("length= " + str(len(posDfs[i])))
    return posDfs   

In [ ]:
def add_negative_events(i):
    negDf = pd.DataFrame(columns = newCols)
    print("i:  " + str(i))
    j = 0
    k = 0
    while(k < len(dfs[i])-300):
        if (j == len(stamps[i]) or (stamps[i][j]*0.2 - k > 300 and ys[i][j] == "car_brake")):
            negDf.loc[len(negDf)] = convert_to_row(int(k), i, "normal")
            print(k)
            k += 300
        else:
            k = stamps[i][j]*0.2 + 600
            j += 1
            while(j < len(stamps[i]) and ys[i][j] != "car_brake"):
                j +=1
    print("length= " + str(len(negDf)))   
    return negDf

In [ ]:
def convert_to_row(start, i, label):
    row = []
    for col in dfs[i].columns:
        row += dfs[i][col][start:start+300].tolist()
    row += [label]
    return row

In [ ]:
fnames = ["VPae", "VPbad", "VPbax", "VPbba", "VPgaa", "VPgab", "VPgac", "VPgae", "VPgag", "VPgah", "VPgal", "VPgam", 
         "VPih", "VPii", "VPja", "VPsaj", "VPsal"]

In [ ]:
for f in fnames:
    print(f)
    files = []
    gc.collect()
    files.append(upload_file("Dataset/" + f + ".mat"))
    dfs = files_to_dfs()
    drop_cols()
    eventsList = events()
    ys = events_to_lists()
    stamps = timestamp_list()
    reacts = reacts_list()
    gc.collect()
    posDf = positive_events()
    posDf[0].to_csv(r"csv/"+"pos"+f+".csv", index = False)
    del posDf
    gc.collect()
    negDf = add_negative_events(0)
    negDf.to_csv(r"csv/"+"neg"+f+".csv", index = False)
    del negDf
    del dfs